In [1]:
import pandas as pd
import csv
from IPython.display import clear_output

In [5]:
def parse_domain(dom_str):

    def sort_domains(domain_list):
        idx = sorted(range(len(domain_list)), key=lambda k: domain_list[k][1][0])
        return [domain_list[i] for i in idx]

    unsorted = [[it[0], list(map(int, it[1].split(':')))] for it in (dom.split(';') for dom in dom_str.split('|'))]

    return sort_domains(unsorted)


def count_real_domains(dom_list):
    return len([dom for dom in dom_list if dom[0]!='d' and dom[0][0]!='m'])


def train_seq_disjoint(dom_list, test_doms):
    return set([dom[0] for dom in dom_list]).isdisjoint(test_doms)

In [3]:
with open('coverage_proteins_UR90.csv') as f:
    with open('coverage_proteins_UR90_no_short.csv', 'w') as fOut:
        for idx, line in enumerate(f):
            if 2000 >= int(line[:-1].split(',')[2]) >= 40:
                fOut.write(line)
            if idx % 1000000 == 0:
                clear_output()
                print(f'Processing Line {idx}')

Processing Line 26000000


In [6]:
from collections import Counter

all_doms = []
with open('coverage_proteins_UR90_no_short.csv') as f:
    for idx, line in enumerate(f):
        all_doms += [dom[0] for dom in parse_domain(line.split(',')[1]) if dom[0]!='d' and dom[0][0]!='m']
        if idx % 1000000 == 0:
            clear_output()
            print(f'Processing Line {idx}')

all_doms = Counter(all_doms)
print(all_doms.most_common(20))
print(len(all_doms))
most_common_doms = set([dom[0] for dom in all_doms.most_common(2000)])
print(most_common_doms)

Processing Line 26000000
[('PF00892', 287551), ('PF03466', 270772), ('PF00126', 261372), ('PF00072', 246007), ('PF00903', 156929), ('PF00440', 146419), ('PF00248', 136479), ('PF00115', 131203), ('PF00571', 130854), ('PF00171', 125080), ('PF00378', 123739), ('PF07992', 121636), ('PF00392', 116550), ('PF00582', 104952), ('PF00441', 95950), ('PF02771', 94349), ('PF02770', 93753), ('PF07690', 93291), ('PF00153', 92865), ('PF01850', 88389)]
19031
{'PF22435', 'PF00561', 'PF00298', 'PF01470', 'PF13548', 'PF07505', 'PF21761', 'PF02548', 'PF01782', 'PF00453', 'PF00576', 'PF01895', 'PF00383', 'PF05960', 'PF00311', 'PF10722', 'PF00142', 'PF00682', 'PF03949', 'PF06250', 'PF01176', 'PF02405', 'PF01156', 'PF13673', 'PF17209', 'PF09838', 'PF06938', 'PF04101', 'PF01850', 'PF03472', 'PF00152', 'PF00111', 'PF01245', 'PF02782', 'PF04402', 'PF22025', 'PF01008', 'PF01927', 'PF14841', 'PF05656', 'PF02586', 'PF01313', 'PF19289', 'PF01609', 'PF01381', 'PF13853', 'PF01172', 'PF02511', 'PF09388', 'PF15919', 'PF

In [16]:
with open('coverage_proteins_UR90_no_short.csv') as f:
    with open('non_test.csv', 'w') as fNonTest:
        with open('coverage_proteins_without_common.csv', 'w') as fOut:
            for idx, line in enumerate(f):
                if set([dom[0] for dom in parse_domain(line.split(',')[1]) if dom[0]!='d']).isdisjoint(most_common_doms):
                    fOut.write(line)
                else:
                    fNonTest.write(line)
                if idx % 1000000 == 0:
                    clear_output()
                    print(f'Processing Line {idx}')

Processing Line 26000000


In [17]:
with open('coverage_proteins_without_common.csv') as f:
    with open('no_idr_proteins_without_common_single_dom.csv', 'w') as fNoIdrSing:
        with open('no_idr_proteins_without_common_multi_dom.csv', 'w') as fNoIdrMulti:
            with open('idr_proteins_without_common_single_dom.csv', 'w') as fIdrSing:
                with open('idr_proteins_without_common_multi_dom.csv', 'w') as fIdrMulti:
                    for idx, line in enumerate(f):
                        dom_list = parse_domain(line.split(',')[1])
                        if ('d' not in [dom[0] for dom in dom_list]) and (count_real_domains(dom_list) == 1):
                            fNoIdrSing.write(line)
                        elif ('d' not in [dom[0] for dom in dom_list]) and (count_real_domains(dom_list) > 1):
                            fNoIdrMulti.write(line)
                        elif ('d' in [dom[0] for dom in dom_list]) and (count_real_domains(dom_list) == 1):
                            fIdrSing.write(line)
                        else:
                            fIdrMulti.write(line)
                        if idx%1000000==0:
                            clear_output()
                            print(f'Processing Line {idx}')

Processing Line 5000000


In [18]:
with open('no_idr_proteins_without_common_single_dom.csv') as f:
    no_idr_sing_counter = 0
    for line in f:
        no_idr_sing_counter += 1
print(no_idr_sing_counter)
with open('no_idr_proteins_without_common_multi_dom.csv') as f:
    no_idr_mult_counter = 0
    for line in f:
        no_idr_mult_counter += 1
print(no_idr_mult_counter)

with open('idr_proteins_without_common_single_dom.csv') as f:
    idr_sing_counter = 0
    for line in f:
        idr_sing_counter += 1
print(idr_sing_counter)
with open('idr_proteins_without_common_multi_dom.csv') as f:
    idr_mult_counter = 0
    for line in f:
        idr_mult_counter += 1
print(idr_mult_counter)

4755875
381487
193326
25045


In [19]:
import numpy as np

no_idr_test_single_dom_ind = set(np.random.choice(a=no_idr_sing_counter, size=100, replace=False))
no_idr_test_multi_dom_ind = set(np.random.choice(a=no_idr_mult_counter, size=2500, replace=False))
idr_test_multi_dom_ind = set(np.random.choice(a=idr_mult_counter, size=250, replace=False))
idr_test_single_dom_ind = set(np.random.choice(a=idr_sing_counter, size=250, replace=False))

In [20]:
test_doms = set()

with open('no_idr_proteins_without_common_single_dom.csv') as f:
    with open('no_idr_single_dom_test_set.csv', 'w') as fTest:
        with open('non_test.csv', 'a') as fNonTest:
            for idx, line in enumerate(f):
                if idx in no_idr_test_single_dom_ind:
                    fTest.write(line)
                    test_doms.update([dom[0] for dom in parse_domain(line.split(',')[1])])
                else:
                    fNonTest.write(line)
                if idx % 1000000 == 0:
                    clear_output()
                    print(f'Processing Line {idx}')

with open('no_idr_proteins_without_common_multi_dom.csv') as f:
    with open('no_idr_multi_dom_test_set.csv', 'w') as fTest:
        with open('non_test.csv', 'a') as fNonTest:
            for idx, line in enumerate(f):
                if idx in no_idr_test_multi_dom_ind:
                    fTest.write(line)
                    test_doms.update([dom[0] for dom in parse_domain(line.split(',')[1])])
                else:
                    fNonTest.write(line)
                if idx % 1000000 == 0:
                    print(f'Processing Line {idx}')

with open('idr_proteins_without_common_single_dom.csv') as f:
    with open('idr_single_dom_test_set.csv', 'w') as fTest:
        with open('non_test.csv', 'a') as fNonTest:
            for idx, line in enumerate(f):
                if idx in idr_test_single_dom_ind:
                    fTest.write(line)
                    test_doms.update([dom[0] for dom in parse_domain(line.split(',')[1])])
                else:
                    fNonTest.write(line)
                if idx % 1000000 == 0:
                    clear_output()
                    print(f'Processing Line {idx}')

with open('idr_proteins_without_common_multi_dom.csv') as f:
    with open('idr_multi_dom_test_set.csv', 'w') as fTest:
        with open('non_test.csv', 'a') as fNonTest:
            for idx, line in enumerate(f):
                if idx in idr_test_multi_dom_ind:
                    fTest.write(line)
                    test_doms.update([dom[0] for dom in parse_domain(line.split(',')[1])])
                else:
                    fNonTest.write(line)
                if idx % 1000000 == 0:
                    print(f'Processing Line {idx}')

Processing Line 0
Processing Line 0


In [21]:
test_ids = set()
with open('no_idr_single_dom_test_set.csv') as f:
    for line in f:
        test_ids.add(line.split(',')[0])
with open('no_idr_multi_dom_test_set.csv') as f:
    for line in f:
        test_ids.add(line.split(',')[0])
with open('idr_single_dom_test_set.csv') as f:
    for line in f:
        test_ids.add(line.split(',')[0])
with open('idr_multi_dom_test_set.csv') as f:
    for line in f:
        test_ids.add(line.split(',')[0])
print(len(test_ids))

3100


In [22]:
test_doms.remove('d')

In [29]:
with open('non_test.csv') as f:
    with open('non_test_nr.csv', 'w') as fOut:
        for idx, line in enumerate(f):
            if set([dom[0] for dom in parse_domain(line.split(',')[1]) if dom[0]!='d']).isdisjoint(test_doms):
                fOut.write(line)
            if idx % 1000000 == 0:
                clear_output()
                print(f'Processing Line: {idx}')

Processing Line: 26000000
